# Extreme boosted trees (XGBoost) for classification (and regression)

Let's take a look at how to train an `xgboost` model for classificaiton in Python using the Titanic dataset from last week. First, load the Titanic data:

In [ ]:
import pandas as pd
import numpy as np # we'll need this later!
import matplotlib.pyplot as plt

# Read the data
df = pd.read_csv('data/titanic.csv')
df.head()

The `xgboost` library is not installed by default in `Anaconda`, so you will need to use `pip` to install it:

In [ ]:
%pip install xgboost

We can now import the xgboost library in the typical way:

In [2]:
import xgboost as xgb

## Data preprocessing and feature selection

When compared to the `RandomForestClassifer()` model, we need to do considerably less preprocessing in advance of model training (e.g., no imputation for missing data). However, `xgboost` will still complain if you try to pass `pandas` "objects" (i.e., strings) directly to your model. As such, let's recode the `Sex` variable to be an integer:

In [3]:
# Preprocess Sex
df['female'] = (df['Sex'] == 'female').astype(int)

Next, in order to compare `xgboost` to our `RandomForestClassifer()` from last week, let's use the same features (**note**: again, you could include all availalble variables in the dataset if you want to!):

In [4]:
# select features
features = ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'female']
y = 'Survived'

### Splitting into training and testing sets

Again, to facilitate comparison, let's split our data into **training** and **testing** sets in the exact same way that we did last week:

In [5]:
# split data into traning and test sets
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)

## XGBoost in Python

We are now ready to fit our model. Given that we are interested in binary classification (survived vs. not survived), we start by setting up an `XGBClassifier` using a `binary:logistic` objective function:

In [6]:
xgb_model = xgb.XGBClassifier(objective='binary:logistic')

As with `sklearn` models, we can train this model (using default hyperparmeters) using the `fit()` method:

In [ ]:
xgb_model.fit(df_train[features], df_train[y])

And we can assess out-of-sample performance in the usual way:

In [ ]:
# Import the metrics from sklearn
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

# Make predictions
y_pred = xgb_model.predict(df_test[features])

# Calculate precision
precision = precision_score(df_test[y], y_pred)
print(f'Precision is {precision}')

# Calculate recall
recall = recall_score(df_test[y], y_pred)
print(f'Recall is {recall}')

# Calculate F1 score
f1 = f1_score(df_test[y], y_pred)
print(f'F1 score is {f1}')

## Hyperparameter tuning and Bayesian optimization

So far, we've used the `XGBClassifier` default hyperparamaters. As with our random forest classifer, it's easy to change these hyperparmeters using the `xgboost` library:

In [ ]:
# Let's lower the learning rate
xgb_model = xgb.XGBClassifier(objective='binary:logistic', learning_rate = .05)

# Fit the model
xgb_model.fit(df_train[features], df_train[y])

# Make predictions
y_pred = xgb_model.predict(df_test[features])

# Calculate precision
precision = precision_score(df_test[y], y_pred)
print(f'Precision is {precision}')

# Calculate recall
recall = recall_score(df_test[y], y_pred)
print(f'Recall is {recall}')

# Calculate F1 score
f1 = f1_score(df_test[y], y_pred)
print(f'F1 score is {f1}')

Wow, that already really helped in terms of performance! We could try to find an even better solution using either `sklearn`'s `GridSearchCV` or `RandomizedSearchCV` as demonstrated last week. However, these approaches are either impossible if you have a large "hyperparmeter space" (`GridSearchCV`) or extremely inefficient (`RandomizedSearchCV`). And the benefit of `xgboost` is it's flexibility: there are many hyperparmeters to choose from in order to find a model suitble for your data. What's a budding data scientist to do?

### Bayesian optimization

The answer: **Bayesian optimization**. The details of Bayesian optimization are quite complex and probably not necessary for us to undertand at this stage. But let's get a basic understanding of how Bayesian optimization  works.

#### What is Bayesian Optimization?
- Used for optimizing expensive-to-evaluate functions (e.g., machine learning hyperparameters).
- Balances **exploration** (searching unknown areas) and **exploitation** (focusing on promising areas).

**Key concepts** for understanding Bayesian optimization:
- **Objective Function**: The function we want to minimize/maximize (e.g., a model's validation error).
- **Probabilistic Model**: A surrogate model (like Gaussian Processes or Tree-structured Parzen Estimators) approximates the objective function.
- **Acquisition Function**: Guides the next point to evaluate by trading off exploration and exploitation.

Bayesian optimization utilizes the following **process**:

1. Start with a handful of initial samples of different hyperparmeters and calculate the performance (e.g., the "function" that you want to optimize) for each combination. (**Note**: you can think of this as a small `RandomizedSearch`.)
2. Fit a model mapping these initial hyperparemeter values to performance. (**Specifically**, the library we use utlizes **Tree-structured Parzen Estimators**.)
3. We then use this model -- contructing what's called an **acuisition function** -- to change our parameters in a way that provides a better overall fit. Note that this is where things start to get really confusing!
4. Repeat until you are satisfied.

The **advantages** of Bayesian optimization over the hyperparameter optimization approaches taht we've discussed so far include:

- Efficient for problems where function evaluations are expensive.
- Reduces the number of evaluations needed compared to grid or random search.

#### Bayesian optimization in Python

The details of Bayesian optimization are less important (at least for us) than the implementation, and the implementation in Python is pretty easy. There are a number of different libraries for Bayesian optimization in Python, but we are going to focus on the `hyperopt` library:

In [ ]:
%pip install hyperopt

Load the necessary functions:

In [10]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll.base import scope # for controlling data types
from sklearn.metrics import f1_score # for evaluating the model

Similar to the `sklearn` functions, we start by setting up a dictionary with our "hyperparameter space":

In [11]:
space = {
    'max_depth': scope.int(hp.quniform('max_depth', 1, 15, 1)),  # Integer values
    'min_child_weight': scope.int(hp.quniform('min_child_weight', 1, 15, 1)),  # Integer values
    'learning_rate': hp.loguniform('learning_rate', -5, -2),  # Float values
    'subsample': hp.uniform('subsample', 0.5, 1),  # Float values
    'colsample_bytree': hp.uniform('colsample_bytree', 0.1, 1),  # Float values
    'n_estimators': scope.int(hp.quniform('n_estimators', 100, 1000, 1))  # Integer values
}

The difference, however, is that we use various [probability distributions](http://hyperopt.github.io/hyperopt/getting-started/search_spaces/) to define the space (instead of hard-coding specific values). We can visualize what each of these distributions looks like using the following:

In [ ]:
from hyperopt.pyll.stochastic import sample

# Function to generate samples from a given hyperparameter distribution
def sample_hyperparameter(hyperparameter, n_samples):
    """
    Generate samples from a given hyperparameter distribution.

    Args:
        hyperparameter: A Hyperopt distribution (e.g., hp.uniform, hp.loguniform).
        n_samples: Number of samples to generate.

    Returns:
        A list of samples from the distribution.
    """
    return [sample(hyperparameter) for _ in range(n_samples)]

# Define the hyperparameter space
space = {
    'max_depth': hp.quniform('max_depth', 1, 15, 1),
    'min_child_weight': hp.quniform('min_child_weight', 1, 15, 1),
    'learning_rate': hp.loguniform('learning_rate', -5, -2),
    'subsample': hp.uniform('subsample', 0.5, 1),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.1, 1),
    'n_estimators': hp.quniform('n_estimators', 100, 1000, 1)
}

# Number of samples to generate
n_samples = 10000

# Sample each hyperparameter using the function
samples = {
    'max_depth': [int(x) for x in sample_hyperparameter(space['max_depth'], n_samples)],
    'min_child_weight': [int(x) for x in sample_hyperparameter(space['min_child_weight'], n_samples)],
    'learning_rate': sample_hyperparameter(space['learning_rate'], n_samples),
    'subsample': sample_hyperparameter(space['subsample'], n_samples),
    'colsample_bytree': sample_hyperparameter(space['colsample_bytree'], n_samples),
    'n_estimators': [int(x) for x in sample_hyperparameter(space['n_estimators'], n_samples)],
}

# Plot the distributions
for param, values in samples.items():
    plt.figure(figsize=(6, 4))
    plt.hist(values, bins=30, color='skyblue', edgecolor='black', alpha=0.7)
    plt.title(f"Distribution of {param}")
    plt.xlabel(param)
    plt.ylabel("Frequency")
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.show()

Next, we need to define the objective function that we want to optimize:

In [13]:
def objective(params):
    # Cast parameters to proper types if necessary
    params['max_depth'] = int(params['max_depth'])
    params['min_child_weight'] = int(params['min_child_weight'])
    params['n_estimators'] = int(params['n_estimators'])
    
    # Define the XGBoost model with the parameters
    xgb_model = xgb.XGBClassifier(
        objective='binary:logistic',
        **params
    )
    
    # Fit the model to the training data
    xgb_model.fit(df_train[features], df_train[y])
    
    # Make predictions
    y_pred = xgb_model.predict(df_test[features])
    
    # Calculate the F1 score (negative because we minimize in Hyperopt)
    score = f1_score(df_test[y], y_pred)
    
    return {'loss': -score, 'status': STATUS_OK}  # Hyperopt minimizes loss


Lastly, we use the `fmin()` function to minimize this objective:

In [ ]:
# Create a Trials object to store intermediate results
trials = Trials()

# Run the Hyperopt optimization
best_params = fmin(
    fn=objective,
    space=space,
    algo=tpe.suggest,
    max_evals=100,
    trials=trials
)

print("Best set of hyperparameters: ", best_params)

Annoyingly, `fmin()` returns floating point numbers for parameters that we need to cast as integers. So we need to quickly turn these floats back into `int`:

In [ ]:
for key in best_params:
    if key in ['max_depth', 'min_child_weight', 'n_estimators']:
        best_params[key] = int(best_params[key])

print("Best set of hyperparameters: ", best_params)

Does this improve our performance?

In [ ]:
# Let's lower the learning rate
xgb_model = xgb.XGBClassifier(objective='binary:logistic', **best_params)

# Fit the model
xgb_model.fit(df_train[features], df_train[y])

# Make predictions
y_pred = xgb_model.predict(df_test[features])

# Calculate precision
precision = precision_score(df_test[y], y_pred)
print(f'Precision is {precision}')

# Calculate recall
recall = recall_score(df_test[y], y_pred)
print(f'Recall is {recall}')

# Calculate F1 score
f1 = f1_score(df_test[y], y_pred)
print(f'F1 score is {f1}')

## Cross-validation with `xgboost`

While the `xgboost` library has a built-in method (i.e., `xgboost.cv()`) for cross-validation, the list of metrics that are available to monitor performance is quite limited and I prefer the flexibility of using `sklearn` to perform cross-validation "manually". Let's see how to do this. First, we need to import the `KFold` function from `sklearn`:

In [17]:
from sklearn.model_selection import KFold

In [ ]:
# Define the classifier
clf = xgb.XGBClassifier(objective='binary:logistic', **best_params)

# Get the k folds
kf = KFold(n_splits=10, shuffle = True, random_state=50)

# Loop over folds and calculate performance measure
results = []
for k, (train_idx, test_idx) in enumerate(kf.split(df[features])):
    # Fit model
    cfit = clf.fit(df[features].iloc[train_idx], df[y].iloc[train_idx])
    
    # Get predictions
    y_pred = cfit.predict(df[features].iloc[test_idx])
    
    # Write results
    result = {'fold': k,
              'precision': precision_score(df[y].iloc[test_idx], y_pred),
              'recall': recall_score(df[y].iloc[test_idx], y_pred),
              'f1': f1_score(df[y].iloc[test_idx], y_pred)}
    # If we want to monitor progress
    print(result)
              
    results.append(result)

In [ ]:
# View results
pd.DataFrame(results)

In [ ]:
# Average precision
np.mean([x['precision'] for x in results])
print(f'Average precision is {np.mean([x["precision"] for x in results])}')

# Average recall
np.mean([x['recall'] for x in results])
print(f'Average recall is {np.mean([x["recall"] for x in results])}')

# Average F1
np.mean([x['f1'] for x in results])
print(f'Average F1 is {np.mean([x["f1"] for x in results])}')

Just like for our decision tree and random forest models, we can plot the most important features for our ``XGBoost`` model:

In [ ]:
# Plot feature importance
xgb.plot_importance(xgb_model, importance_type='weight')  # 'weight' is the default
plt.title('Feature Importance')
plt.show()

## Using our final model in "production"

So we've have a model that we think is pretty good -- now what? We can now use our model to predict new data by, for instance, creating a "would you have survived the Titanic app". The steps for resusing our "final" model include:

1. Fit the final model using **all** the data.
2. `pickle` the model for later use

And then when you want to predict a new observation, you:

3. Read in the data and format it **exactly** how the training data was formatted. For us, that means reading in our data and storing it as a `pandas` dataframe with the following variables: ['Survived', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'female'].

First, we fit and save the model:

In [21]:
# Define the classifier
clf = xgb.XGBClassifier(objective='binary:logistic', **best_params)

# Fit on all data
cfit = clf.fit(df[features], df[y])

# Save the model
import pickle
pickle.dump(cfit, open('xgb_model.pkl', 'wb'))

We can then load the model as follows:

In [22]:
loaded_model = pickle.load(open('xgb_model.pkl', 'rb'))

In [ ]:
# Create a new example observation as a dictionary with the variable names as keys
new_obs = {'PassengerId': 1,
            'Survived': 0,
            'Pclass': 3,
            'Name': 'Braverman, Suella',
            'Sex': 'Female', 
            'Age': 43.0,
            'SibSp': 0,
            'Parch': 0,
            'Ticket': 'A/5 21171',
            'Fare': 7.25,
            'Cabin': '',
            'Embarked': 'S',
            'female': 1}

# Convert to a dataframe
df_new_obs = pd.DataFrame([new_obs])

# Make a prediction
prob = loaded_model.predict_proba(df_new_obs[features])
print(f'Probability of survival is {prob[0][1]}')


## Regression using `xgboost`

Solving regression problems with `xgboost` follow the same basic syntax. Let's start by importing our housing price data that we used to demonstrate regression analysis in Python:

In [ ]:
data = pd.read_csv('data/housing.csv')
data.head()

And split the data into training and testing sets:

In [ ]:
data_train, data_test = train_test_split(data, test_size=.3, random_state=42)
data_train.head()

In [26]:
from xgboost import XGBRegressor

Grab some features:

In [27]:
y = 'SalePrice'
features = ['LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'GrLivArea', 'FullBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'GarageCars', 'GarageArea']

In [ ]:
data[features].dtypes

Setup an `XGBREgressor()` model:

In [29]:
model = XGBRegressor(objective='reg:squarederror')

Train the model and get out of sample predictions:

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# Fit the model
model.fit(data_train[features], data_train[y])

# Make predictions
y_pred = model.predict(data_test[features])

# Calculate RMSE
rmse = np.sqrt(mean_squared_error(data_test[y], y_pred))
print(f'RMSE is {rmse}')

# Calculate R^2
r2 = r2_score(data_test[y], y_pred)
print(f'R^2 is {r2}')

# Calculate MAE
mae = mean_absolute_error(data_test[y], y_pred)
print(f'MAE is {mae}')

Which features are the most important in our model?

In [ ]:
# Plot feature importance
xgb.plot_importance(model, importance_type='weight')  # 'weight' is the default
plt.title('Feature Importance')
plt.show()

That's it!

### You try

Can you use Bayesian optimization to find the best hyperparameters for our real estate regression model? Complete the following steps:

1. Determine how your "features" (or variables) are measured. What type of probability distribution can you use to represent each variable?
2. Set up your "sample" space using `hp`.
3. Define your objective function. What is a good measure of performance for regression problems?
4. Use `fmin()` to find the best parameters and print them.
5. Fit the model using the best parameters. Does the out-of-sample performance increase?
